In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
from nlpsig_networks.pytorch_utils import SaveBestModel, training_pytorch, testing_pytorch, set_seed, KFold_pytorch

In [3]:
import os
os.chdir("../../")

In [4]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

Rumours

In [5]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [8]:
x_data = obtain_SWNUNetwork_input(
    method = "umap",
    dimension= 30,
    df= df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings.numpy(),
    k=5,
    time_feature='time_encoding',
    standardise_method=None,
    add_time_in_path=False)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 415])

SWNU Network

Obtaining path by looking at post history
We can obtain a path by looking at the history of each post. Here we look at the last 20 posts (and pad with vectors of zeros if there are less than 20 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [9]:
#dimensionality reduction
embedding_dim = sbert_embeddings.shape[1]
dim_reduce_method = ["umap"] #["gaussian_random_projection", "umap"]
dimensions = [30]#[50,30,15]
#time features
time_features = "time_encoding"
standardise_method = "standardise"
add_time_in_path = False
#SWNU block
augmentation_tp = "Conv1d"
hidden_dim_aug = None
comb_m = "concatenation"
log_sig = True
conv_output_channels = [10] #[20, 10, 5]
log_signature_dimensions_and_sig_depths = [(8, 3)]#[(30, 2), (10, 3), (6, 4)]
bidirectional = False
#ffn
hidden_dim_sizes = [64]#[32,64]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
#overall training
num_epochs = 100
batch=64
patience = 4
learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [10]:
size=20
swnu_network_log_signature, best_swnu_network_log_signature, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=dim_reduce_method,
    dimensions=dimensions, 
    sig_depths=[x[1] for x in log_signature_dimensions_and_sig_depths], 
    log_signature=log_sig,
    conv_output_channels=conv_output_channels,
    swnu_hidden_dim_sizes=[x[0] for x in log_signature_dimensions_and_sig_depths],
    ffn_hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    time_feature=time_features, 
    standardise_method=standardise_method,
    add_time_in_path = add_time_in_path,
    augmentation_type = augmentation_tp,
    hidden_dim_aug = hidden_dim_aug,
    comb_method = comb_m,
    path_indices=None,
    data_split_seed=123,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 30 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

- average (validation) metric score: 0.591780816450916
scores for the different seeds: [0.626982463494725, 0.584666795193111, 0.5117702231037717, 0.6024335031126202, 0.6330510973503517]
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [11]:
swnu_network_log_signature

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,k,dimensions,...,dropout_rate,learning_rate,seed,BiLSTM,gamma,k_fold,augmentation_type,hidden_dim_aug,comb_method,model_id
0,focal,0.553861,0.525842,"[0.6411042944785276, 0.41057934508816124]",0.207095,0.661922,0.626982,"[0.7411444141689373, 0.5128205128205128]",20,30,...,0.2,0.0001,0,False,2,False,Conv1d,None,concatenation,0
0,focal,0.585319,0.553508,"[0.672686230248307, 0.4343302990897269]",0.216002,0.590747,0.584667,"[0.6349206349206349, 0.5344129554655871]",20,30,...,0.2,0.0001,1,False,2,False,Conv1d,None,concatenation,0
0,focal,0.626311,0.623477,"[0.656140350877193, 0.5908141962421711]",0.234876,0.597865,0.511770,"[0.7167919799498748, 0.3067484662576687]",20,30,...,0.2,0.0001,12,False,2,False,Conv1d,None,concatenation,0
0,focal,0.568160,0.531348,"[0.6626954579300074, 0.4]",0.219130,0.644128,0.602434,"[0.7311827956989247, 0.47368421052631576]",20,30,...,0.2,0.0001,123,False,2,False,Conv1d,None,concatenation,0
0,focal,0.588179,0.552814,"[0.6785714285714286, 0.4270557029177719]",0.216144,0.669039,0.633051,"[0.7479674796747967, 0.5181347150259067]",20,30,...,0.2,0.0001,1234,False,2,False,Conv1d,None,concatenation,0


In [12]:
swnu_network_log_signature.groupby(["k",
                                    "dimensions",
                                    "sig_depth",
                                    "method",
                                    "lstm_hidden_dim",
                                    "ffn_hidden_dim",
                                    "dropout_rate",
                                    "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_39300/2753258485.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_log_signature.groupby(["k",


,,,,,,,,accuracy,f1,valid_loss,valid_accuracy,valid_f1,input_channels,output_channels,add_time_in_path,num_time_features,embedding_dim,log_signature,seed,BiLSTM,gamma,k_fold,model_id
k,dimensions,sig_depth,method,lstm_hidden_dim,ffn_hidden_dim,dropout_rate,learning_rate,,,,,,,,,,,,,,,,
20,30,3,umap,8,64,0.2,0.0001,0.584366,0.557398,0.218649,0.63274,0.591781,30.0,10.0,0.0,1.0,384.0,1.0,274.0,0.0,2.0,0.0,0.0


In [13]:
best_swnu_network_log_signature

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,k,dimensions,...,ffn_hidden_dim,dropout_rate,learning_rate,seed,BiLSTM,gamma,k_fold,augmentation_type,hidden_dim_aug,comb_method
0,focal,0.553861,0.525842,"[0.6411042944785276, 0.41057934508816124]",0.207095,0.661922,0.626982,"[0.7411444141689373, 0.5128205128205128]",20,30,...,64,0.2,0.0001,0,False,2,False,Conv1d,None,concatenation
0,focal,0.585319,0.553508,"[0.672686230248307, 0.4343302990897269]",0.216002,0.590747,0.584667,"[0.6349206349206349, 0.5344129554655871]",20,30,...,64,0.2,0.0001,1,False,2,False,Conv1d,None,concatenation
0,focal,0.626311,0.623477,"[0.656140350877193, 0.5908141962421711]",0.234876,0.597865,0.511770,"[0.7167919799498748, 0.3067484662576687]",20,30,...,64,0.2,0.0001,12,False,2,False,Conv1d,None,concatenation
0,focal,0.568160,0.531348,"[0.6626954579300074, 0.4]",0.219130,0.644128,0.602434,"[0.7311827956989247, 0.47368421052631576]",20,30,...,64,0.2,0.0001,123,False,2,False,Conv1d,None,concatenation
0,focal,0.588179,0.552814,"[0.6785714285714286, 0.4270557029177719]",0.216144,0.669039,0.633051,"[0.7479674796747967, 0.5181347150259067]",20,30,...,64,0.2,0.0001,1234,False,2,False,Conv1d,None,concatenation
